# 0. GPU 확인

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2982397699668712677,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23399170048
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6334106490859937404
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"]

# 1. 이미지 전처리

## 1) path 설정

### train_data

In [2]:
train_data_original_img_path = './data/UNET/img_1000/resize_512/train_data/original_img'
train_data_resized_img_path = './data/UNET/img_1000/resize_512/train_data/resized_img'

### test_data

In [3]:
test_data_original_img_path = './data/UNET/img_1000/resize_512/test_data/original_img'
test_data_resized_img_path = './data/UNET/img_1000/resize_512/test_data/resized_img'

## 2) 이미지 목록 확인

In [3]:
import glob
import os
from os import listdir, makedirs
from os.path import isfile, join

In [4]:
def find_img_name_list(original_img_path) :
    img_name_list = list(filter(lambda f: isfile(join(original_img_path, f)), listdir(original_img_path)))
    
    return img_name_list

### train_data

In [5]:
train_data_img_name_list = find_img_name_list(train_data_original_img_path)

In [6]:
print(len(train_data_img_name_list))
print(train_data_img_name_list[:3])

988
['ILSVRC2012_val_00001157.jpg', 'ILSVRC2012_val_00030120.jpg', 'ILSVRC2012_val_00001857.jpg']


### test_data

In [7]:
test_data_img_name_list = find_img_name_list(test_data_original_img_path)

In [8]:
print(len(test_data_img_name_list))
print(test_data_img_name_list[:3])

99
['ILSVRC2012_val_00040557.jpg', 'ILSVRC2012_val_00002728.jpg', 'ILSVRC2012_val_00025052.jpg']


## 3) 흑백 이미지 삭제

In [10]:
from PIL import Image
import os
import numpy as np
import time
from IPython.display import clear_output

In [11]:
def remove_gray_img(img_name_list, original_img_path) :
    start = time.time()

    i = 0
    for img_name in img_name_list:
        
        clear_output(wait=True)
        
        original_img = Image.open(os.path.join(original_img_path, img_name))
        pix = np.array(original_img)
        if pix.ndim != 3 :
            os.remove(os.path.join(original_img_path, img_name))
            
        i += 1
        print('진행 상황 : ', i,'/',len(img_name_list))
        print('진행 시간 : ', round((time.time() - start), 2))

    print('총 걸린 시간 : ', round((time.time() - start),2), '초')

### train_data

In [12]:
remove_gray_img(train_data_img_name_list, train_data_original_img_path)
train_data_img_name_list = list(filter(lambda f: isfile(join(train_data_original_img_path, f)), listdir(train_data_original_img_path)))

진행 상황 :  988 / 988
진행 시간 :  6.46
총 걸린 시간 :  6.46 초


### test_data

In [13]:
remove_gray_img(test_data_img_name_list, test_data_original_img_path)
test_data_img_name_list = list(filter(lambda f: isfile(join(test_data_original_img_path, f)), listdir(test_data_original_img_path)))

진행 상황 :  99 / 99
진행 시간 :  0.25
총 걸린 시간 :  0.25 초


## 4) 원본 이미지의 사이즈 저장

In [14]:
from PIL import Image
import pickle

In [15]:
def make_original_img_size_list(img_name_list, original_img_path) :
    original_img_size_list = []
    for img_name in img_name_list:
        original_img = Image.open(os.path.join(original_img_path, img_name))
        original_img_size_list.append(original_img.size)
    
    return original_img_size_list

In [17]:
def save_object_by_pickle(obj, file_path):
    pickle.dump(obj, open(file_path, 'wb'), protocol=4)

### test_data (만 필요) 

In [18]:
test_data_original_img_size_list = make_original_img_size_list(test_data_img_name_list, test_data_original_img_path)

In [19]:
print(len(test_data_original_img_size_list))
print(test_data_original_img_size_list[:3])

99
[(300, 450), (300, 450), (300, 340)]


In [20]:
test_data_original_img_size_path = './data/UNET/img_1000/resize_512/test_data/original_img_size.pkl'

In [21]:
save_object_by_pickle(test_data_original_img_size_list, test_data_original_img_size_path)

## 5) resized 및 저장

In [22]:
import cv2
import time
from IPython.display import clear_output
import os

In [23]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [24]:
def resizing_img(img_name_list, original_img_path, resized_img_path, resize_size) :
    start = time.time()

    i = 0
    for img_name in img_name_list :

        clear_output(wait=True)


        try:
            original_img = cv2.imread(os.path.join(original_img_path, img_name))
            resized_img = cv2.resize(original_img, dsize = resize_size, interpolation = cv2.INTER_CUBIC)
            createDirectory(resized_img_path)
            cv2.imwrite(os.path.join(resized_img_path, img_name), resized_img)
        except:
            print('{}가 변환되지 않았습니다.'.format(img_name))


        i += 1
        print('진행 상황 : ', i,'/',len(img_name_list))
        print('진행 시간 : ', round((time.time() - start), 2))

    print('총 걸린 시간 : ', round((time.time() - start),2), '초')

### train_data

In [25]:
resizing_img(train_data_img_name_list, train_data_original_img_path, train_data_resized_img_path, (512, 512))

진행 상황 :  988 / 988
진행 시간 :  10.18
총 걸린 시간 :  10.18 초


### test_data

In [26]:
resizing_img(test_data_img_name_list, test_data_original_img_path, test_data_resized_img_path, (512, 512))

진행 상황 :  99 / 99
진행 시간 :  0.6
총 걸린 시간 :  0.6 초


## 6) 이미지 RGB 값 pickle 저장

In [1]:
import numpy as np
import time
from IPython.display import clear_output
from PIL import Image
from tensorflow.keras import datasets, utils, backend

import glob
import os
from os import listdir, makedirs
from os.path import isfile, join

import pickle

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def load_my_img(img_name_list, resized_img_path) :
    start = time.time()

    i = 0
    
    img_pixel_list = [0]*len(img_name_list)
    for img_name in img_name_list :

        clear_output(wait=True)


        resized_img = Image.open(os.path.join(resized_img_path, img_name))
        pixel_array = np.array(resized_img)
        pixel_list = pixel_array.tolist()
        img_pixel_list[i] = pixel_list

        i += 1
        print('진행 상황 : ', i,'/',len(img_name_list))
        print('진행 시간 : ', round((time.time() - start), 2))

    print('총 걸린 시간 : ', round((time.time() - start),2), '초')
    
    img = np.array(img_pixel_list)
    
    return img

In [3]:
def save_object_by_pickle(obj, file_path):
    pickle.dump(obj, open(file_path, 'wb'), protocol=4)

In [4]:
def find_img_name_list(original_img_path) :
    img_name_list = list(filter(lambda f: isfile(join(original_img_path, f)), listdir(original_img_path)))
    
    return img_name_list

### train_data

In [5]:
train_data_original_img_path = './data/UNET/img_1000/resize_512/train_data/original_img'
train_data_resized_img_path = './data/UNET/img_1000/resize_512/train_data/resized_img'

In [6]:
train_data_img_name_list = find_img_name_list(train_data_original_img_path)

In [7]:
train_img_pixel_values = load_my_img(train_data_img_name_list, train_data_resized_img_path)

진행 상황 :  988 / 988
진행 시간 :  87.38
총 걸린 시간 :  87.38 초


In [8]:
train_data_img_pixel_values_path = './data/UNET/img_1000/resize_512/train_data/img_pixel_values.pkl'

In [9]:
save_object_by_pickle(train_img_pixel_values, train_data_img_pixel_values_path)

### test_data

In [10]:
test_data_original_img_path = './data/UNET/img_1000/resize_512/test_data/original_img'
test_data_resized_img_path = './data/UNET/img_1000/resize_512/test_data/resized_img'

In [11]:
test_data_img_name_list = find_img_name_list(test_data_original_img_path)

In [12]:
test_img_pixel_values = load_my_img(test_data_img_name_list, test_data_resized_img_path)

진행 상황 :  99 / 99
진행 시간 :  8.37
총 걸린 시간 :  8.37 초


In [13]:
test_data_img_pixel_values_path = './data/UNET/img_1000/resize_512/test_data/img_pixel_values.pkl'

In [14]:
save_object_by_pickle(test_img_pixel_values, test_data_img_pixel_values_path)

In [15]:
raise SystemExit("Stop right there!")

SystemExit: Stop right there!

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 여기까지 실행

# 여기서부터 다시 시작

# 2. 학습

## 1) 이미지 RGB 값 pickle 불러오기

In [ ]:
import pickle
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
def load_object_by_pickle(pklfile):
    return pickle.load(open(pklfile, 'rb'))

In [ ]:
train_data_img_pixel_values_path = './data/UNET/img_1000/resize_512/train_data/img_pixel_values.pkl'

In [ ]:
train_img = load_object_by_pickle(train_data_img_pixel_values_path)

## 2) Train Dataset 구축

In [ ]:
from tensorflow.keras import backend

In [ ]:
class DATA():
    def __init__(self, img_data) :

        if backend.image_data_format() == 'channels_first':
            n_ch, img_rows, img_cols = img_data.shape[1:]
            input_shape = (1, img_rows, img_cols)
        else:
            img_rows, img_cols, n_ch = img_data.shape[1:]
            input_shape = (img_rows, img_cols, 1)

        img_data = img_data.astype('float32') / 255.0
    
        def RGB2Gray(img, fmt):
            if fmt == 'channels_first':
                R = img[:, 0:1]
                G = img[:, 1:2]
                B = img[:, 2:3]
            else:
                R = img[..., 0:1]
                G = img[..., 1:2]
                B = img[..., 2:3]
            return 0.299 * R + 0.587 * G + 0.114 * B
    
        x = RGB2Gray(img_data, backend.image_data_format())
        
        self.input_shape = input_shape
        self.x, self.y = x, img_data
        self.n_ch = n_ch

In [ ]:
data_train = DATA(train_img)
print(data_train.input_shape, data_train.x.shape)

In [ ]:
X_train = data_train.x
Y_train = data_train.y

## 3) 모델 구축

In [ ]:
from tensorflow.keras import models, backend
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, BatchNormalization, Concatenate, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def conv(x, n_f, mp_flag=True):
    x = MaxPooling2D((2, 2), padding='same')(x) if mp_flag else x
    x = Conv2D(n_f, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.05)(x)
    x = Conv2D(n_f, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def deconv_unet(x, e, n_f, ic):
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(n_f, (2, 2), padding='same')(x)
    x = Concatenate(axis=ic)([x, e])
    x = Conv2D(n_f, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(n_f, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def UNET(org_shape, n_ch):
    ic = 3 if backend.image_data_format() == 'channels_last' else 1
    # Input
    original = Input(shape=org_shape)

    # Encoding
    e1 = conv(original, 64, mp_flag=False)
    e2 = conv(e1, 128)
    e3 = conv(e2, 256)
    e4 = conv(e3, 512)

    encoded = conv(e4, 1024)

    # Decoding
    x = deconv_unet(encoded, e4, 512, ic)
    x = deconv_unet(x, e3, 256, ic)
    x = deconv_unet(x, e2, 128, ic)
    x = deconv_unet(x, e1, 64, ic)

    decoded = Conv2D(n_ch, (1, 1), padding='same')(x)


    model = Model(inputs=original, outputs=decoded)
    optimizer = Adam(lr=0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['acc'])

    return model

In [ ]:
unet = UNET(data_train.input_shape, data_train.n_ch)
unet.summary()

## 5) Callback 설정

In [ ]:
import tensorflow.keras
import os

In [ ]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [ ]:
epochs=100
batch_size=8

img_num = 1000
resize_size = 512

In [ ]:
model_save_folder = './data/UNET/img_1000/resize_512/trained_model'
model_filename_front = model_save_folder + '/unet_model-img_num_{}-resize_size_{}-batch_size_{}-'.format(img_num, resize_size, batch_size)
model_filename = model_filename_front + '{epoch}-{val_loss:.4f}-{loss:.4f}.h5'

In [ ]:
createDirectory(model_save_folder)

In [ ]:
earlystopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(model_filename, minitor='val_loss', mode='min', save_best_only=True)

## 6) 모델 학습 

In [ ]:
history = unet.fit(X_train, Y_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   validation_split=0.2,
                   callbacks=[earlystopping, checkpoint])

## 7) 학습 loss 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_acc(history, title=None):
    # summarize history for accuracy
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    if title is not None:
        plt.title(title)
    plt.ylabel('Accracy')
    plt.xlabel('Epoch')
    plt.legend(['Training data', 'Validation data'], loc=0)
    # plt.show()


def plot_loss(history, title=None):
    # summarize history for loss
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training data', 'Validation data'], loc=0)
    # plt.show()


def plot_history(history):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plot_acc(history)
    plt.subplot(1, 2, 2)
    plot_loss(history)

In [ ]:
plot_loss(history)

In [ ]:
raise SystemExit("Stop right there!")

# 여기까지 실행

# 여기서부터 다시 시작

# 3. 예측

## 1) Test DataSet 구축

In [ ]:
from tensorflow.keras import backend
import ssl
import pickle
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
class DATA():
    def __init__(self, img_data) :

        if backend.image_data_format() == 'channels_first':
            n_ch, img_rows, img_cols = img_data.shape[1:]
            input_shape = (1, img_rows, img_cols)
        else:
            img_rows, img_cols, n_ch = img_data.shape[1:]
            input_shape = (img_rows, img_cols, 1)

        img_data = img_data.astype('float32') / 255.0
    
        def RGB2Gray(img, fmt):
            if fmt == 'channels_first':
                R = img[:, 0:1]
                G = img[:, 1:2]
                B = img[:, 2:3]
            else:
                R = img[..., 0:1]
                G = img[..., 1:2]
                B = img[..., 2:3]
            return 0.299 * R + 0.587 * G + 0.114 * B
    
        x = RGB2Gray(img_data, backend.image_data_format())
        
        self.input_shape = input_shape
        self.x, self.y = x, img_data
        self.n_ch = n_ch

In [ ]:
def load_object_by_pickle(pklfile):
    return pickle.load(open(pklfile, 'rb'))

In [ ]:
test_data_img_pixel_values_path = './data/UNET/img_1000/resize_512/test_data/img_pixel_values.pkl'

In [ ]:
test_img = load_object_by_pickle(test_data_img_pixel_values_path)

In [ ]:
data_test = DATA(test_img)
print(data_test.input_shape, data_test.x.shape)

In [ ]:
X_test = data_test.x
Y_test_real = data_test.y

## 2) 학습된 모델 불러오기

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_path = './data/UNET/img_1000/resize_512/trained_model/unet_model-img_num_1000-resize_size_512-batch_size_8-20-0.0074-0.0098.h5'

In [ ]:
trained_model  = load_model(model_path)

## 3) 예측 진행

In [ ]:
Y_test_predicted = trained_model.predict(X_test)

## 4) 예측 결과 시각화

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import minmax_scale

In [ ]:
def show_images(x, y_real, y_predicted):
    if backend.image_data_format() == 'channels_first':
        print(y_real.shape)
        y_real = y_real.swapaxes(1, 3).swapaxes(1, 2)
        print(y_real.shape)
        y_predicted = y_predicted.swapaxes(1, 3).swapaxes(1, 2)

        x = x[:, 0, ...]

    else:
        x = x[..., 0]
        
    n = 10
    plt.figure(figsize=(20, 6))
    for i in range(n):

        ax = plt.subplot(3, n, i + 1)
        if x.ndim < 4:
            plt.imshow(x[i], cmap='gray')
        else:
            plt.imshow(x[i])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(y_predicted[i])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n * 2)
        plt.imshow(y_real[i])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

In [ ]:
show_images(X_test, Y_test_real, Y_test_predicted)

## 4) 원본 크기와 이름 불러오기

In [ ]:
import glob
import os
from os import listdir, makedirs
from os.path import isfile, join

In [ ]:
test_data_original_img_path = './data/UNET/img_1000/resize_512/test_data/original_img'
test_data_resized_img_path = './data/UNET/img_1000/resize_512/test_data/resized_img'
test_data_img_name_list = list(filter(lambda f: isfile(join(test_data_original_img_path, f)), listdir(test_data_original_img_path)))

In [ ]:
test_data_original_img_size_path = './data/UNET/img_1000/resize_512/test_data/original_img_size.pkl'
test_data_original_img_size = load_object_by_pickle(test_data_original_img_size_path)

## 5) 예측된 이미지 size 복원 후 저장

In [ ]:
import cv2
import time
from IPython.display import clear_output

In [ ]:
def resize_and_save_restored_img(img_name_list, decoded_imgs, original_img_size, restored_img_path) :
    start = time.time()
    
    for i in range(len(img_name_list)) :

        clear_output(wait=True)

        restored_img_name = img_name_list[i]

        try:
            resized_restored_img = cv2.resize(decoded_imgs[i], dsize = original_img_size[i], interpolation = cv2.INTER_CUBIC)
            resized_restored_img = resized_restored_img*255.0
            resized_restored_img = resized_restored_img.astype('int64')
            cv2.imwrite(os.path.join(restored_img_path, restored_img_name), resized_restored_img)

        except:
            print('{}가 변환되지 않았습니다.'.format(restored_img_name))

        i += 1
        print('진행 상황 : ', i,'/',len(img_name_list))
        print('진행 시간 : ', round((time.time() - start), 2))

    print('총 걸린 시간 : ', round((time.time() - start),2), '초')

In [ ]:
def load_object_by_pickle(pklfile):
    return pickle.load(open(pklfile, 'rb'))

In [ ]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [ ]:
restored_img_path = './data/UNET/img_1000/resize_512/test_data/restored_img'

In [ ]:
createDirectory(restored_img_path)

In [ ]:
resize_and_save_restored_img(test_data_img_name_list, Y_test_predicted, test_data_original_img_size, restored_img_path)